In [1]:
import tensorflow as tf

2024-02-28 21:14:20.767886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 21:14:21.345634: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-28 21:14:21.348179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-28 21:14:22.625046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import numpy as np

dtype = tf.float16 # subject to change
BASE_DIR = "../../phi_local/weights/"
DECODER_LAYERS = 4
params = {}
params['embed_tokens'] = tf.constant(np.load(BASE_DIR + 'embed_tokens.npy'), dtype=dtype)
params['decoder_layers'] = []

for i in range(DECODER_LAYERS):
  layer_params = {}
  layer_params['layernorm_weight'] = 'layernorm_weight.npy'
  layer_params['layernorm_bias'] = 'layernorm_bias'
  layer_params['q_proj_weight'] = 'q_proj_weight'
  layer_params['q_proj_bias'] = 'q_proj_bias'
  layer_params['k_proj_weight'] = 'k_proj_weight'
  layer_params['k_proj_bias'] = 'k_proj_bias'
  layer_params['v_proj_weight'] = 'v_proj_weight'
  layer_params['v_proj_bias'] = 'v_proj_bias'
  layer_params['dense_weight'] = 'dense_weight'
  layer_params['dense_bias'] = 'dense_bias'
  layer_params['mlp_fc1_weight'] = 'mlp_fc1_weight'
  layer_params['mlp_fc1_bias'] = 'mlp_fc1_bias'
  layer_params['mlp_fc2_weight'] = 'mlp_fc2_weight'
  layer_params['mlp_fc2_bias'] = 'mlp_fc2_bias'
  for key in layer_params:
    layer_params[key] = tf.constant(np.load(BASE_DIR + str(i) + '_' + key + '.npy'), dtype=dtype)
    if "weight" in key and "layernorm" not in key:
      layer_params[key] = tf.transpose(layer_params[key], perm=[1,0])
  params['decoder_layers'].append(layer_params)

params['final_layernorm_weight'] = tf.constant(np.load(BASE_DIR + 'final_layernorm_weight.npy'), dtype=dtype)
params['final_layernorm_bias'] = tf.constant(np.load(BASE_DIR + 'final_layernorm_bias.npy'), dtype=dtype)

params['lm_head_weight'] = tf.transpose(tf.constant(np.load(BASE_DIR + 'lm_head_weight.npy'), dtype=dtype), perm=[1,0])
params['lm_head_bias'] = tf.constant(np.load(BASE_DIR + 'lm_head_bias.npy'), dtype=dtype)
  

2024-02-28 21:14:25.144704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-28 21:14:25.146500: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
class PhiConfig():
    keys_to_ignore_at_inference = ["past_key_values"]
    def __init__(
        self,
        vocab_size=51200,
        hidden_size=2560,
        intermediate_size=10240,
        num_hidden_layers=DECODER_LAYERS, # modified,
        num_attention_heads=32,
        num_key_value_heads=32,
        resid_pdrop=0.0, # the model bt default has 0.1 on colab, but we dont have randomness
        embd_pdrop=0.0,
        attention_dropout=0.0,
        hidden_act="gelu_new",
        max_position_embeddings=2048,
        initializer_range=0.00,
        layer_norm_eps=1e-5,
        use_cache=True,                  # Modifed
        tie_word_embeddings=False,
        rope_theta=10000.0,
        rope_scaling=None,
        partial_rotary_factor=0.4,
        qk_layernorm=False,
        bos_token_id=50256,
        eos_token_id=50256,
    ):
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.intermediate_size = intermediate_size
        self.num_hidden_layers = num_hidden_layers
        self.num_attention_heads = num_attention_heads

        if num_key_value_heads is None:
            num_key_value_heads = num_attention_heads

        self.num_key_value_heads = num_key_value_heads
        self.resid_pdrop = resid_pdrop
        self.embd_pdrop = embd_pdrop
        self.attention_dropout = attention_dropout
        self.hidden_act = hidden_act
        self.max_position_embeddings = max_position_embeddings
        self.initializer_range = initializer_range
        self.layer_norm_eps = layer_norm_eps
        self.use_cache = use_cache
        self.rope_theta = rope_theta
        self.rope_scaling = rope_scaling
        self.partial_rotary_factor = partial_rotary_factor
        self.qk_layernorm = qk_layernorm
        self.pad_token_id = None
        self._attn_implementation = 'eager'

In [4]:
import my_phi_gm
config_gm = PhiConfig()
import importlib

In [69]:
importlib.reload(my_phi_gm)
importlib.reload(my_phi_gm.bert)
importlib.reload(my_phi_gm.masking_utils_gm)
importlib.reload(my_phi_gm.cache)
config_gm = PhiConfig(use_cache=True)
my_model_gm = None
my_model_gm = my_phi_gm.PhiForCausalLM(config_gm, params)
# testing input to see if graphmode works
input_ids_gm = tf.constant([[40, 588, 836, 5500, 13, 1867, 318, 534, 4004, 2057, 30]])
mask_gm = tf.constant([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
model_kwargs_gm = {'use_cache': True, 'attention_mask': mask_gm}
model_inputs_gm = my_model_gm.prepare_inputs_for_generation(input_ids_gm, **model_kwargs_gm)
out_gm = my_model_gm(
        **model_inputs_gm,
        return_dict=False, # important
        output_attentions=False,
        output_hidden_states=False
        )
print(out_gm[0])

past_key_values before for loop []
past_key_values []
past_key_value in decoder: []
past key value_old in decoder layer: []
past_key_value after tf.identity: []
layer_id:  0
past_key_value: []
past_key_value after appending: [[<tf.Tensor 'concat_1:0' shape=(1, 32, 11, 80) dtype=float16>, <tf.Tensor 'transpose_2:0' shape=(1, 32, 11, 80) dtype=float16>]]
done updating
next decoder cache in loop iteration: [[<tf.Tensor 'PartitionedCall_3:1' shape=(1, 32, 11, 80) dtype=float16>, <tf.Tensor 'PartitionedCall_3:2' shape=(1, 32, 11, 80) dtype=float16>]]
past_key_values []
past_key_value in decoder: [[<tf.Tensor 'PartitionedCall_3:1' shape=(1, 32, 11, 80) dtype=float16>, <tf.Tensor 'PartitionedCall_3:2' shape=(1, 32, 11, 80) dtype=float16>]]
past key value_old in decoder layer: [[<tf.Tensor 'past_key_value_old:0' shape=(1, 32, 11, 80) dtype=float16>, <tf.Tensor 'past_key_value_old_1:0' shape=(1, 32, 11, 80) dtype=float16>]]
past_key_value after tf.identity: [[<tf.Tensor 'past_key_value_old:0' s

In [72]:
# testing input to see if graphmode works

import my_phi
importlib.reload(my_phi)
importlib.reload(my_phi.bert)
importlib.reload(my_phi.masking_utils)
importlib.reload(my_phi.cache)
my_model = None
my_model = my_phi.PhiForCausalLM(config, params)
input_ids = tf.constant([[40, 588, 836, 5500, 13, 1867, 318, 534, 4004, 2057, 30]])
mask = tf.constant([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
model_kwargs = {'use_cache': True, 'attention_mask': mask}
model_inputs = my_model.prepare_inputs_for_generation(input_ids, **model_kwargs)
out = my_model(
        **model_inputs,
        return_dict=True,
        output_attentions=False,
        output_hidden_states=False
        )
print(out.logits)
# testing an self-attntion-layer
# input = tf.random.uniform([1, 11, 2560], dtype=tf.float16)
# attn = None
# pos_ids = tf.range([11])
# print(pos_ids)
# attn = my_phi.PhiAttention(config, params['decoder_layers'][0], 0)
# attn(input, pos_ids)

position_ids: tf.Tensor([[ 0  1  2  3  4  5  6  7  8  9 10]], shape=(1, 11), dtype=int32)
mask: tf.Tensor(
[[[[     0. -65504. -65504. -65504. -65504. -65504. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0. -65504. -65504. -65504. -65504. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0. -65504. -65504. -65504. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0. -65504. -65504. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0.      0. -65504. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0.      0.      0. -65504. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0.      0.      0.      0. -65504.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0.      0.      0.      0.      0.
    -65504. -65504. -65504.]
   [     0.      0.      0.      0.      0.      0.      0.      0.
         0. -65504. -65504.]
   [     0.      0. 

In [56]:
import math

@tf.function
def test(input):
    x = math.sqrt(3)
    return tf.nn.softmax((1 - tf.experimental.numpy.triu(input, k=3)) / x)
    # print(tf.math.reduce_max(input).shape)

@tf.function
def test2(input):
    x1 = test(input)
    print(x1)
    return [[tf.cast(x1, dtype=tf.float16), x1], x1]

@tf.function
def test3(input):
    return 1 + test2(input)[0][1]

@tf.function
def test4(i1, i2):
    if i2 is None:
        i2 = tf.ones([4,4])
    i1 += 2
    return i1, i2

@tf.function
def test5(i1, i2):
    x1, x2 = test4(i1, i2)
    x2 = [tf.ones([4,4]), tf.ones([4,4])]
    x2[0] += 1
    return test4(x1, x2)

@tf.function
def test6(i1):
    i2 = [i[:] for i in i1] # allows us to create a deep copy of a list(list(tensors))
    # i2 = i1[:]
    # print(i2)
    i2[1][0] += i2[1][0]
    return i2

x = [[tf.zeros([4,]),], [tf.ones([4,]),]]
# x = [tf.zeros([4,]),]
out = test6(x)
print(out)
# print(x)
# print(y)

[[<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>], [<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 2., 2., 2.], dtype=float32)>]]
